In [ ]:
%load_ext autoreload
%autoreload 2

## Load Portfolio including stock symbols and desired networth distribution

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

pd.set_option('display.max_rows', 1000)

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# https://developers.google.com/sheets/api/guides/authorizing
credentials = ServiceAccountCredentials.from_json_keyfile_name(
         'google-sheets-credentials.json', scope)

gc = gspread.authorize(credentials)

wks = gc.open("Portfolio").sheet1

data = wks.get_all_values()
headers = data.pop(0)

df = pd.DataFrame(data, columns=headers)
df[["Shares", "Purchase Price (EUR)", "Desired Distribution"]] = df[["Shares", "Purchase Price (EUR)", "Desired Distribution"]].apply(pd.to_numeric)
df = df.drop(['Ticker', 'Accumulating'], axis=1)
display(df)

## Get stock prices in EUR
EUR is my national currency.

In [ ]:
from portfolio import stock_price_eur
        
stock_price = pd.Series([stock_price_eur(r) for r in df['Symbol']], name='Stock Price')
df = pd.concat([df, stock_price], axis=1)
display(df)

## Calculate current distribution
The current distribution is based on the current price of the stocks.

In [ ]:
from portfolio import current_distribution

df = pd.concat([df, current_distribution(df)], axis=1)
display(df)

## Calculate investment plan
It is possible to exclude stocks from the plan manually.
They are excluded automatically when their price is higher than the calculated investment.
Each stock has a weight in the distribution. The weight of excluded stocks are spread over the remaining ones leading to a new distribution and investment plan.

In [ ]:
from portfolio import calc_shares_to_purchase
    
res = calc_shares_to_purchase(df, 4000, order_cost=7, exclusions=['X501.DE'])
networth = (res['Stock Price'] * res['Shares'] + res['To Invest']).sum()
res['Expected Distribution'] = (res['Stock Price'] * res['Shares'] + res['To Invest']) / networth
sum_row = res.filter(['Purchase Price ', 'Desired Distribution', 'Current Distribution', 'New Relative Distribution', 'To Invest', 'Expected Distribution']).sum().rename('Total')
res = res.append(sum_row)
res = res.sort_values(by=['Sort Order'])
display(res.filter(['ISIN', 'Category', 'Stock Price', 'To Invest', 'Shares To Purchase', 'Expected Distribution']))